## LPC 8xx Programming Tool

SPSDK provides command line utility *lpcprog* that might be used for in system programming of LPC devices. In this example it will be demostrated on LPCXpresso860-MAX board. ISP protocol is described in [AN13815](https://www.nxp.com/docs/en/application-note/AN13815.pdf) document.

This example shows the basic operation using the SPSDK CLI application "lpcprog".

## 1. Prerequisites
- SPSDK is needed with examples extension. `pip install spsdk[examples]` (Please refer to the [installation](../_knowledge_base/installation_guide.rst) documentation.)

- This example uses LPC800 EVK board.

### 1.1 Hardware setup

When the ISP entry pin (PIO0_12) is pulled LOW on reset, the part enters ISP mode and
the ISP command handler starts up.
PIO0_25 is UART0 TX, and PIO0_24 is UART0 RX, are used for USART ISP mode.

In this example we have used FTDI TTL232-3V3 USB to serial converter.

![board-connection](img/board_connection.jpg "board-connection")

### 1.2 Synchronization

All ISP responses are sent as CR-LF terminated ASCII strings. Data is sent and received in plain binary format.
The host sends a '?' (0x3F) as a synchronization character and waits for a response. The host side serial port settings are 8 data bits, one stop bit, and no parity. 

The auto-baud routine programs the baud rate of
the serial port generator by measuring the bit time of the received synchronization character in terms of its frequency. It also sends an ASCII string (Synchronized) to the host. In response to this action, the host sends back the same string (Synchronized CR-LF).

The auto-baud routine looks at the received characters to verify synchronization. If synchronization is verified, the Synchronized OK string is sent to the host (“echo” is enabled by default). The host responds by sending the crystal frequency (in kHz) at which the part is running. The response is required
for backward compatibility of the bootloader code and is ignored. 

The OK string is sent to the host after receiving the crystal frequency. If synchronization is not verified, the auto-baud routine waits again for a synchronization character.
Once the crystal frequency is received, the part initializes, and the ISP command handler invokes. For safety reasons, an "Unlock" command is required before executing the commands resulting in flash erase/write operations and the "Go" command. The rest of the commands are executed without the unlock command. Execute the "Unlock" command once per ISP session.

In [29]:
# Initialization

# This env variable sets colored logger output to STDOUT
%env JUPYTER_SPSDK=1
# Set a magic for command execution and echo
%alias execute echo %l && %l
%alias_magic ! execute

WORKSPACE = "workspace/"  # change this to path to your workspace
DATA = "data/"  # data folder that contains input images
VERBOSITY = (
    "-v"  # verbosity of commands, might be -v or -vv for debug or blank for no additional info
)
# PORT = "-p /dev/tty.usbserial-FTCEE1TG"  # change this to your port
PORT = "-p COM126"

# Synchronize
%! lpcprog $PORT $VERBOSITY sync

env: JUPYTER_SPSDK=1
Created `%!` as an alias for `%execute`.
lpcprog -p /dev/tty.usbserial-FTCEE1TG -v sync
INFO:spsdk.lpcprog.interface:Synchronized
Synchronized


After successfull synchronization, we can try other commands.

## 2. CLI Operations
### 2.1 Get info command
This command is used for getting information about the chip.
It reports part ID, UID, boot code version and CRP level. 

In [31]:
# Get information about the device
%! lpcprog $PORT $VERBOSITY get-info

lpcprog -p /dev/tty.usbserial-FTCEE1TG -v get-info
INFO:spsdk.lpcprog.protocol:CMD: read_uid, STATUS: Success
INFO:spsdk.lpcprog.protocol:CMD: read_boot_code_version, STATUS: Success
INFO:spsdk.lpcprog.protocol:CMD: read_part_id, STATUS: Success
INFO:spsdk.lpcprog.protocol:CMD: read_memory, STATUS: Success
Part ID: LPC865M201JBD64d
UID: 0x0301001f 0x95cc1007 0x61aa9196 0xf5000282
Boot code version: 0.240
CRP Level: NO_CRP
All USART ISP commands are supported.


### 2.2 Read memory
This command is used for reading flash and RAM. User might specify file --binary where the content will be stored.

In [32]:
# Read the flash memory
%! lpcprog $PORT $VERBOSITY read-memory 0 0x100

lpcprog -p /dev/tty.usbserial-FTCEE1TG -v read-memory 0 0x100
Reading memory  [------------------------------------]    0%INFO:spsdk.lpcprog.protocol:CMD: read_memory, STATUS: Success
Reading memory  [####################################]  100%
00000000: E0 1F 00 10 E9 00 00 00  5F 01 00 00 61 05 00 00  ........_...a...
00000010: 00 00 00 00 00 00 00 00  00 00 00 00 77 D8 FF EF  ............w...
00000020: 00 00 00 00 00 00 00 00  00 00 00 00 63 01 00 00  ............c...
00000030: 00 00 00 00 00 00 00 00  65 01 00 00 C9 04 00 00  ........e.......
00000040: 6B 01 00 00 73 01 00 00  7B 01 00 00 83 01 00 00  k...s...{.......
00000050: 8B 01 00 00 93 01 00 00  9B 01 00 00 A3 01 00 00  ................
00000060: AB 01 00 00 B3 01 00 00  BB 01 00 00 C3 01 00 00  ................
00000070: CB 01 00 00 D3 01 00 00  DB 01 00 00 E3 01 00 00  ................
00000080: EB 01 00 00 F3 01 00 00  FB 01 00 00 03 02 00 00  ................
00000090: 0B 02 00 00 13 02 00 00  1B 02 00 00 23 02 00 00  ..

In [33]:
# Read from RAM
%! lpcprog $PORT $VERBOSITY read-memory 0x1000_0800 0x100

lpcprog -p /dev/tty.usbserial-FTCEE1TG -v read-memory 0x1000_0800 0x100
Reading memory  [------------------------------------]    0%INFO:spsdk.lpcprog.protocol:CMD: read_memory, STATUS: Success
Reading memory  [####################################]  100%
00000000: 70 58 94 06 D8 AF BA 7E  5F 0D FE 55 30 A9 55 DC  pX.....~_..U0.U.
00000010: 4E BA F1 8E 00 4C EA 38  6F 69 93 24 6E C1 25 AD  N....L.8oi.$n.%.
00000020: 65 42 02 32 72 C8 7D D4  F1 82 3E 58 E3 5F 30 DD  eB.2r.}...>X._0.
00000030: A1 A0 E5 63 F8 D9 7A CD  AE DA 16 9B 34 36 F0 C4  ...c..z.....46..
00000040: DC 02 8F 8A EA 2D 48 EF  DA A5 A8 BA 5A F1 13 06  .....-H.....Z...
00000050: 25 9D 1F C0 9F DB 05 1E  3F 86 16 4D 28 D7 B4 85  %.......?..M(...
00000060: C1 83 87 AD 40 34 D3 F3  27 E4 BB AB 59 80 75 3D  ....@4..'...Y.u=
00000070: A1 4E A0 87 81 AB AD FC  19 AE 10 11 54 CA 4E 5F  .N..........T.N_
00000080: C9 6C 8B 8A D2 0F 47 1E  5F FB B1 D6 5C E2 74 5D  .l....G._...\.t]
00000090: A7 58 9D 84 12 C0 60 7B  23 16 7A 11 07 2A

### 2.3 Compare memory
This command is used to compare the memory contents at two locations.

In [34]:
# Compare the content of the flash memory
%! lpcprog $PORT $VERBOSITY compare 0x10 0x20 4

lpcprog -p /dev/tty.usbserial-FTCEE1TG -v compare 0x10 0x20 4
INFO:spsdk.lpcprog.protocol:CMD: compare, STATUS: Success

Status: Success
Description: Success
Content is same


### 2.4 Erase flash memory

Flash memory might be erased by sectors or by pages.

In [35]:
# Erase memory from sector 0x10 to sector 0x20
%! lpcprog $PORT $VERBOSITY erase-sector 0x10 0x20

lpcprog -p /dev/tty.usbserial-FTCEE1TG -v erase-sector 0x10 0x20
INFO:spsdk.lpcprog.protocol:CMD: unlock, STATUS: Success
INFO:spsdk.lpcprog.protocol:CMD: prepare_sectors_for_write, STATUS: Success
INFO:spsdk.lpcprog.protocol:CMD: erase_sector, STATUS: Success

Status: Success
Description: Success


### 2.5 Check blank sectors
This command is used to blank check one or more sectors of on-chip flash memory

In [36]:
# Check blank sectors
%! lpcprog $PORT $VERBOSITY blank-check-sectors 0x10 0x20

lpcprog -p /dev/tty.usbserial-FTCEE1TG -v blank-check-sectors 0x10 0x20
INFO:spsdk.lpcprog.protocol:CMD: blank_check_sectors, STATUS: Success

Status: Success
Description: Success
Sectors are blank


### 2.6 Write to RAM
This command is used for writing to RAM, user might specify file or binary pattern using the syntax "{{}}" 

In [37]:
# Write pattern to RAM
%! lpcprog $PORT $VERBOSITY write-ram -a 0x1000_0800 -b [[deadbeef]]

lpcprog -p COM126 -v write-ram -a 0x1000_0800 -b [[deadbeef]] 
INFO:spsdk.lpcprog.protocol:CMD: write_ram, STATUS: Success
Data has been written to RAM


### 2.7 Read CRC checksum
This command is used to read the CRC checksum of a block of RAM or flash memory.

In [38]:
# Get CRC-32 checksum
%! lpcprog $PORT $VERBOSITY read-crc-checksum 0 128

lpcprog -p /dev/tty.usbserial-FTCEE1TG -v read-crc-checksum 0 128
INFO:spsdk.lpcprog.protocol:CMD: read_crc_checksum, STATUS: Success
Checksum: 0x7fa26eca


### 2.8 Read flash signature
This command is used to read the flash signature generated by the flash controller.

In [39]:
# Get flash signature
%! lpcprog $PORT $VERBOSITY read-flash-signature 0 128

lpcprog -p /dev/tty.usbserial-FTCEE1TG -v read-flash-signature 0 128
INFO:spsdk.lpcprog.protocol:CMD: read_flash_signature, STATUS: Success
Signature: 0x44d83974


### 2.9 Flash programming

This command is used for programming the flash sector.

Approach for writing the sector
* Write data to RAM
* Prepare sector for writing
* Erase sector
* Again prepare sector
* Copy RAM to flash

In this example we are going to write "led blinky" demo application that blinks with green on-board LED.

This command requires information about the device, so the correct sector and page size and flash range is correctly obtained. If the device ID cannot be decoded, it must be provided as -f/--family option.

In [40]:
# Write LED blinky application
%! lpcprog -f lpc865 $PORT $VERBOSITY program-flash -b "../_data/lpcxpresso860_led_blinky.bin"

lpcprog -f lpc865 -p /dev/tty.usbserial-FTCEE1TG -v program-flash -b ../_data/lpcxpresso860_led_blinky.bin
Programming flash memory  [------------------------------------]    0%INFO:spsdk.lpcprog.protocol:CMD: unlock, STATUS: Success
INFO:spsdk.lpcprog.protocol:Writing flash sector: 0 Address: 0x0
INFO:spsdk.lpcprog.protocol:CMD: write_ram, STATUS: Success
INFO:spsdk.lpcprog.protocol:Preparing sector 0 for erase
INFO:spsdk.lpcprog.protocol:CMD: prepare_sectors_for_write, STATUS: Success
INFO:spsdk.lpcprog.protocol:Erasing sector 0
INFO:spsdk.lpcprog.protocol:CMD: erase_sector, STATUS: Success
INFO:spsdk.lpcprog.protocol:Preparing sector 0 for write
INFO:spsdk.lpcprog.protocol:CMD: prepare_sectors_for_write, STATUS: Success
INFO:spsdk.lpcprog.protocol:Copying 1024B RAM 0x10000800 to flash address 0x0
INFO:spsdk.lpcprog.protocol:CMD: copy_ram_to_flash, STATUS: Success
INFO:spsdk.lpcprog.protocol:Calculating checksum
INFO:spsdk.lpcprog.protocol:CMD: read_crc_checksum, STATUS: Success
INFO

### 2.10 Baudrate change

Using the command set-baud-rate it is possible to change baudrate

In [41]:
# Set baudrate to 460800
%! lpcprog $PORT $VERBOSITY set-baud-rate -r 460800

lpcprog -p /dev/tty.usbserial-FTCEE1TG -v set-baud-rate -r 460800
INFO:spsdk.lpcprog.protocol:CMD: set_baud_rate, STATUS: Success

Status: Success
Description: Success


In [42]:
# Now we have to provide the new baudrate as the -b/--baudrate command
%! lpcprog -b 460800 $PORT $VERBOSITY get-info

lpcprog -b 460800 -p /dev/tty.usbserial-FTCEE1TG -v get-info
INFO:spsdk.lpcprog.protocol:CMD: read_uid, STATUS: Success
INFO:spsdk.lpcprog.protocol:CMD: read_boot_code_version, STATUS: Success
INFO:spsdk.lpcprog.protocol:CMD: read_part_id, STATUS: Success
INFO:spsdk.lpcprog.protocol:CMD: read_memory, STATUS: Success
Part ID: LPC865M201JBD64d
UID: 0x0301001f 0x95cc1007 0x61aa9196 0xf5000282
Boot code version: 0.240
CRP Level: NO_CRP
All USART ISP commands are supported.
